# MicroNet Challenge (Team: OSI AI)

Our team build a network having `X` accuracy on cifar-100 with `A` parameters and `B` multiply-add operations, achieveing the MicroNet Challenge score of `xx`.

## 1. Overview
The below figure is our proposed architecture for the cifar-100 dataset. The numbers described above the arrows are the shape of each input and output.  
Our architecture consists of:  
1. Upsample Layer
2. Stem_Conv
3. 10 $\times$ MobileNet V2 Convolution Block (MBConvBlock)
4. Head_Conv
5. Global Average Pooling
6. Fully Connected Layer  

The details of Stem_Conv, Head_Conv, and MBConvBlock are described below the 'Main network'.
* In addition, in MBConvBlock\[0\], there is no the first three layers (Expansion_Conv, BatchNorm, Activation Function) in a block since there is no expansion when $e=1$.

<img src="./figure/overview_v1.png" width="1200"/>

## 2. Our Approach Detail

### 2-0. Configuration
* <b>Data & Model precision</b>
    * 16 bits
* <b>Data (Please refer to `Config/main.json`)</b>
    * Dataset: cifar-100
    * Batch size: 128
    * Train size/Valid size: 50000/0
    * Augmentation: \[random crop 32*32 with padding of 4, random horizontal flip(p=0.5), normalization\] \+ (custom) auto augmentation for cifar-100 \+ Mixup
* <b>Model (Please refer to `Config/main.json`)</b>
    * Architecture: See `figure 1`
    * Activation function: swish (beta=1)
    * Batch normalization: ghost batch normalization (splits=4)
    * Optimizer: sgd (lr=0.1, weight_decay=1e-5, momentum=0.9)
    * Loss function: cross entropy loss with label smoothing (smoothing factor=0.3)
    * Learning rate scheduler: cosine annealing scheduler (T_max=1200, without restart)
    * Epochs #: 1200
* <b>Pruning (Please refer to `Config/pruning.json`)</b>
    * Pruning method(one shot/iterative): iterative
    * Desired sparsity/Pruning ratio per iteration: 50%/10%
    * Epochs # per pruning iteration: 300
    * Optimizer: sgd (lr=0.1, weight_decay=1e-5, momuntum=0.9)
    * Loss function: cross entropy loss with label smoothing (smoothing factor=0.3)
    * Learning rate scheduler: cosine annealing scheduler (T_max=300, without restart)

### 2-1. Architecture Search
First of all, we search for a baseline architecture suitable for cifar-100 data set based on the [EfficientNet](https://arxiv.org/pdf/1905.11946.pdf) architecture using autoML. The search process is as follows:
1. <b>Block arguments search</b>: In this step, we search the number of MBConvBlock, and kernel size(k), stride(s), expansion ratio(e), input channels(i), output channels(o), and squeeze-expansion ratio(se) in each block. From the results of the block arguments search, we find out that the position of the convolutional layer which serves to reduce resolution, or convolutional layer with stride of 2, is a sensitive factor to accuracy. With this inference, after several hand-made experiments, the above architecture is chosen. (Here, the number of channels differs from the above figure because scaling coefficients are not yet considered.)

2. <b>Scaling coefficients search</b>: In this step, after block aurgments are decided, we search three coefficients by adjusting available resources: width, depth, and resolution. Actually, we set the depth coefficient as 1 since its slight change gets even worse in terms of score. Therefore, a resolution coefficient is set randomly within a given range according to the available resources, and then a width coefficient is calculated by \[available resources / resolution coefficient$^2$\].  From the results of the scaling coefficients search, we find out that a large resolution coefficient make a greater performance improvement than a large width coefficient under our circumstance. As a result, when we set available resources as 2, we get a resolution coefficient of 1.4. Finally, to lighten this model, we decide a width coefficient as 0.9, and adapt these coefficients to the model we've got via block arguments search.

### 2-2. Techniques for Improvement
* <b>[Auto augmentation](https://arxiv.org/pdf/1805.09501.pdf)</b>: We search 25 sub-policies for cifar-100 data set based on the augmentation search space in `AutoAugment` except `Cutout` and `SamplePairing`. Please refer to `AutoML_autoaug.py` for the process and `data_utils/autoaugment.py` for the policy we've got.
* <b>[Mixup](https://arxiv.org/pdf/1710.09412.pdf)</b>: We add a Mixup technique with $\alpha$ of 1, which is the hyperparameter for beta-distribution, after auto augmentation. It is believed that this augmentation can help inter-exploration between arbitrary two classes.
* <b>[No bias decay](https://arxiv.org/pdf/1812.01187.pdf)</b>: We do not apply weight decay regularizer to biases. Since these part has a small percentage of the total, it can make underfitting.
* <b>[Swish activation function](https://arxiv.org/pdf/1710.05941.pdf)</b>: We use a <i>Swish</i> activation function with $\beta$ of 1, which is $x\times sigmoid(x)$. This activation function is usually interpreted as a self-gate activation.
* <b>[Ghost batch normalization](https://arxiv.org/pdf/1705.08741.pdf)</b>: We use ghost batch normalization, where batch is divided into four smaller ghost batch in our case to match the splited batch size to 32, instead of plain batch normalization.
* <b>[Label smoothing](https://arxiv.org/pdf/1512.00567.pdf)</b>: We use a label smoothing technique through which the probability of the correct label is assinged as 0.7, and $\frac{0.3}{99}$ for the others.
* <b>[Cosine annealing scheduler](https://arxiv.org/pdf/1608.03983.pdf)</b>: We use cosine annealing scheduler for adaptive learning rate, and set a period of one cycle as the number of epochs. Hence, there is no restart process.

The results are as follows. 점진적으로 technique 추가하면서 table로 결과 표기

### 2-3. Pruning
After training the main network, we adapt magnitude-based iterative pruning method. We prune 10% from whole weights and repeat 5 times in the same way, and hence 50% of whole parameters are pruned.  
The trade-off between accuracy and sparsity is as follows.

### 2-4. Early Exiting
Although general CNN models have the static computational graph for whole dataset, many samples can be classified with few computations.  
By exiting certain samples ealiear, considerable FLOPs can be saved without significant accuracy degradation.  

To decide the best trade-off exiting position, we checked out three candidates in our models; MBConvBlock\[2\], MBConvBlock\[3\], and MBConvBlock\[4\].  
Considering the trade-off between accuracy and FLOPs, we finally chose MBConvBlock\[2\].  

To ensure the performance of the main network to be preserved, <b>we freeze the pruned model and update the parameters of early exiting module only</b>.
* <b>Data</b>
    * Same with the data configuration for the main network
    * Excpet not using (custom) auto augment & Mixup
* <b>Model</b>
    * Same with the optimization configuration for the main network
    * Except not using label smoothing, epochs # is 400, and using adapted loss function.
    
: early exiting 비율 / final 결과 / 
특정 블럭에서 나왔을 때, 어느정도의 flops compared with~  
Note that the results are not obtained from validation results.

In [83]:
63853343 + 60203448

124056791

## 3. Scoring metric
| <div style="width:70px">Input</div>      | Operator         |  k  |  s  |  e  |  i  |  o  |  se  | Parameter Storage | MULTI      |  ADD       | Math Operations |
| :---                                     | :---:            |:---:|:---:|:---:|:---:|:---:| :---:| :---:             | :---:      | :---:      | :---:           |
| $32^{2}\times3$                          | Upsample(nearest)| -   | -   | -   | -   | -   | -    | 0                 | 11,907     | 0          | 11,907          |
| $63^{2}\times3$                          | Stem\_Conv2d     | 3   | 2   | -   | 3   | 24  | -    | 648               | 691,920    | 622,728    | 1,314,648       |
| $31^{2}\times24$                         | MBConvBlock\[0\] | 3   | 1   | 1   | 24  | 16  | 0.20 | 424               | 669,132    | 584,456    | 1,253,588       |
| $31^{2}\times16$                         | MBConvBlock\[1\] | 3   | 1   | 6   | 16  | 24  | 0.20 | 6,192             | 5,170,329  | 4,593,288  | 9,763,617       |
| $31^{2}\times24$                         | MBConvBlock\[2\] | 3   | 2   | 6   | 24  | 40  | 0.20 | 13,056            | 5,462,148  | 4,940,136  | 10,402,284      |
| $15^{2}\times40$                         | MBConvBlock\[3\] | 3   | 1   | 6   | 40  | 40  | 0.20 | 35,040            | 5,207,904  | 4,873,800  | 10,081,704      |
| $15^{2}\times40$                         | MBConvBlock\[4\] | 3   | 1   | 6   | 40  | 48  | 0.20 | 35,088            | 5,639,904  | 5,304,000  | 10,943,904      |
| $15^{2}\times48$                         | MBConvBlock\[5\] | 3   | 1   | 6   | 48  | 64  | 0.20 | 49,632            | 8,328,267  | 7,923,744  | 16,252,011      |
| $15^{2}\times64$                         | MBConvBlock\[6\] | 3   | 1   | 6   | 64  | 64  | 0.20 | 86,784            | 12,501,348 | 11,966,784 | 24,468,132      |
| $15^{2}\times64$                         | MBConvBlock\[7\] | 3   | 2   | 6   | 64  | 80  | 0.20 | 86,880            | 7,598,436  | 7,277,104  | 14,875,540      |
| $7^{2}\times80$                          | MBConvBlock\[8\] | 3   | 1   | 6   | 80  | 80  | 0.20 | 135,360           | 4,233,408  | 4,086,160  | 8,319,568       |
| $7^{2}\times80$                          | MBConvBlock\[9\] | 3   | 1   | 6   | 80  | 96  | 0.20 | 135,456           | 4,609,728  | 4,461,696  | 9,071,424       |
| $7^{2}\times96$                          | Head\_Conv2d     | 1   | 1   | -   | 96  | 136 | -    | 13,056            | 659,736    | 639,744    | 1,299,480       |
| $7^{2}\times136$                         | AveragePool      | 7   | -   | -   | -   | -   | -    | 0                 | 136        | 6,528      | 6,664           |
| $136$                                    | FullyConnected   | -   | -   | -   | -   | -   | _    | 13,600            | 13,600     | 13,500     | 27,100          |
| $100$                                    | -                | -   | -   | -   | -   | -   | -    | -                 | -          | -          | -               |
| $15^{2}\times40$                         | EarlyExiting     | -   | -   | -   | -   | -   | -    | 30,920            | 3,055,440  | 2,909,780  | 5,965,220       |
| Total                                    | -                | -   | -   | -   | -   | -   | -    | 642,136           | 63,853,343 | 60,203,448 | 124,056,791     |

In [92]:
earlyExitingOps = 11907+1314648+1253588+9763617+10402284+10081704+5965220
notEarlyExitingOps = 124056791

earlyExitingRatio = 0.3
print (int(earlyExitingRatio*earlyExitingOps+(1-earlyExitingRatio)*notEarlyExitingOps))

98477644


### 3-1. Parameter Storage

* Upsample(nearest): $0$
* Stem_Conv2d: $3 \times 3 \times 3 \times 24$ (Conv2d) + $0$ (BatchNorm) + $0$ (Activation) = $648$
* MBConvBlock(k,s,e,i,o,se): $1 \times 1 \times i \times (i \times e)$ (Expansion_Conv) + $k \times k \times (i \times e)$ (Depthwise_Conv) + $1 \times 1 \times (i \times e) \times (i \times e \times se)$ (SE_squeeze) + $1 \times 1 \times (i \times e \times se) \times (i \times e)$ (SE_expand) +  $1 \times 1 \times e \times o$ (Projection_conv)
    * when $e=1$, Expansion_Conv is omitted. (i.e., $- 1 \times 1 \times i \times (i \times e)$)
* Head_Conv2d: $1 \times 1 \times 96 \times 136$ (Conv2d) + $0$ (BatchNorm) + $0$ (Activation) = $13,056$
* AveragePool: $0$
* FullyConnected:$136 \times 100$ = $13,600$
* Early Exisiting: $1 \times 1 \times i \times 2i$ (Pointwise_Conv) + $3 \times 3 \times 2i$ (Depthwise_Conv) + $1 \times 1 \times 2i \times 150$ (Projection_Conv) + $150 \times 100$ (Fully connected)

In [47]:
params = 0

# Stem_Conv
params += 3*3*3*24

blocks = [[3, 1, 1, 24, 16, 0.20],
          [3, 1, 6, 16, 24, 0.20],
          [3, 2, 6, 24, 40, 0.20],
          [3, 1, 6, 40, 40, 0.20],
          [3, 1, 6, 40, 48, 0.20],
          [3, 1, 6, 48, 64, 0.20],
          [3, 1, 6, 64, 64, 0.20],
          [3, 2, 6, 64, 80, 0.20],
          [3, 1, 6, 80, 80, 0.20],
          [3, 1, 6, 80, 96, 0.20]]

for k, s, e, i, o, se in blocks:
    block_params = 0
    if e!=1:
        block_params += 1*1*i*(i*e)
    block_params += k*k*(i*e)
    block_params += 1*1*(i*e)*int(i*e*se)
    block_params += 1*1*int(i*e*se)*(i*e)
    block_params += 1*1*e*o
    params += block_params
params += 1*1*96*136
params += 0
params += 136*100

early_exiting_in_channels = 40
early_exiting_out_channels = 150
params += 1*1*early_exiting_in_channels*(2*early_exiting_in_channels) + 3*3*(2*early_exiting_in_channels) +\
          1*1*(2*early_exiting_in_channels)*early_exiting_out_channels + early_exiting_out_channels*100

print (params)

642136


### 3-2. Math Operations

* Upsample(nearest): $63\times63\times3=11,907$
* Stem_Conv2d: $3 \times 3 \times 3 \times 24$ (Conv2d) + $0$ (BatchNorm) + $0$ (Activation) = $648$
* MBConvBlock(k,s,e,i,o,se): $(1 \times 1 \times i) \times (w_{in} \times h_{in} \times (i \times e))$
    * when $e=1$, Expansion_Conv is omitted. (i.e., $- 1 \times 1 \times i \times (i \times e)$)
* Head_Conv2d:
* AveragePool:
* FullyConnected:
* Early Exisiting:
    * When early exiting happens,
    * When early exiting does not happen, early exiting + 뒷부분

In [65]:
total_mul_ops = 0
total_add_ops = 0

# Upsample
total_mul_ops += 63*63*3
total_add_ops += 0

# Stem_Conv (Conv2d)
tmp_mul_ops = (3*3*3)*(31*31*24)
tmp_add_ops = (3*3*3-1)*(31*31*24)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

# Stem_Conv (Swish Activation)
tmp_mul_ops = (3)*(31*31*24)
tmp_add_ops = (1)*(31*31*24)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

blocks = [[31, 31, 3, 1, 1, 24, 16, 0.20],
          [31, 31, 3, 1, 6, 16, 24, 0.20],
          [31, 15, 3, 2, 6, 24, 40, 0.20],
          [15, 15, 3, 1, 6, 40, 40, 0.20],
          [15, 15, 3, 1, 6, 40, 48, 0.20],
          [15, 15, 3, 1, 6, 48, 64, 0.20],
          [15, 15, 3, 1, 6, 64, 64, 0.20],
          [15,  7, 3, 2, 6, 64, 80, 0.20],
          [ 7,  7, 3, 1, 6, 80, 80, 0.20],
          [ 7,  7, 3, 1, 6, 80, 96, 0.20]]

for idx, (input_size, output_size, k, s, e, i, o, se) in enumerate(blocks):
    mul_ops = 0
    add_ops = 0
    
    # Expansion
    if e!=1:
        tmp_mul_ops = (1*1*i)*(input_size*input_size*(i*e))
        tmp_add_ops = (1*1*i-1)*(input_size*input_size*(i*e))
        mul_ops += tmp_mul_ops
        add_ops += tmp_add_ops
    # Swish Activation
    if e!=1:
        tmp_mul_ops = (3)*(input_size*input_size*(i*e))
        tmp_add_ops = (1)*(input_size*input_size*(i*e))
        mul_ops += tmp_mul_ops
        add_ops += tmp_add_ops
    # Depthwise
    tmp_mul_ops = (k*k)*(output_size*output_size*(i*e))
    tmp_add_ops = (k*k-1)*(output_size*output_size*(i*e))
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    # Swish Activation
    tmp_mul_ops = (3)*(output_size*output_size*(i*e))
    tmp_add_ops = (1)*(output_size*output_size*(i*e))
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    
    # SE module (GlobalAvgPool)
    tmp_mul_ops = i*e
    tmp_add_ops = (output_size*output_size-1)*(i*e)
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    # SE module (SE_squeeze)
    tmp_mul_ops = (1*1*(i*e))*(1*1*int(i*e*se))
    tmp_add_ops = (1*1*(i*e)-1)*(1*1*int(i*e*se))
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    # SE module (Swish Activation)
    tmp_mul_ops = (3)*(1*1*int(i*e*se))
    tmp_add_ops = (1)*(1*1*int(i*e*se))
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    # SE module (SE_expand)
    tmp_mul_ops = (1*1*int(i*e*se))*(1*1*(i*e))
    tmp_add_ops = (1*1*int(i*e*se)-1)*(1*1*(i*e))
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    # SE module (Sigmoid)
    tmp_mul_ops = (2)*(1*1*(i*e))
    tmp_add_ops = (1)*(1*1*(i*e))
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops
    
    # Channel-wise product
    tmp_mul_ops = output_size*output_size*(i*e)
    tmp_add_ops = 0
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops  
    # Projection
    tmp_mul_ops = (1*1*(i*e))*(output_size*output_size*o)
    tmp_add_ops = (1*1*(i*e)-1)*(output_size*output_size*o)
    mul_ops += tmp_mul_ops
    add_ops += tmp_add_ops  
    
    total_mul_ops += mul_ops
    total_add_ops += add_ops
    
# Head_Conv (Conv2d)
tmp_mul_ops = (1*1*96)*(7*7*136)
tmp_add_ops = (1*1*96-1)*(7*7*136)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

# Head_Conv (Swish Activation)
tmp_mul_ops = (3)*(7*7*136)
tmp_add_ops = (1)*(7*7*136)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

# GlobalAvgPool
tmp_mul_ops = 136
tmp_add_ops = (7*7-1)*136
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

# FullyConnected
tmp_mul_ops = 136*100
tmp_add_ops = (136-1)*100
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

# EarlyExiting (Pointwise)
tmp_mul_ops = (1*1*40)*(15*15*80)
tmp_add_ops = (1*1*40-1)*(15*15*80)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (Swish Activation)
tmp_mul_ops = (3)*(15*15*80)
tmp_add_ops = (1)*(15*15*80)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (Depthwise)
tmp_mul_ops = (3*3)*(13*13*80)
tmp_add_ops = (3*3-1)*(13*13*80)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (Swish Activation)
tmp_mul_ops = (3)*(13*13*80)
tmp_add_ops = (1)*(13*13*80)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (Projection)
tmp_mul_ops = (1*1*80)*(13*13*150)
tmp_add_ops = (1*1*80-1)*(13*13*150)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (Swish Activation)
tmp_mul_ops = (3)*(13*13*150)
tmp_add_ops = (1)*(13*13*150)
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (GlobalAvgPool)
tmp_mul_ops = 150
tmp_add_ops = (13*13-1)*150
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops
# EarlyExiting (FullyConnected)
tmp_mul_ops = 150*100
tmp_add_ops = (150-1)*100
total_mul_ops += tmp_mul_ops
total_add_ops += tmp_add_ops

print (total_mul_ops, total_add_ops)

63853343 60203448


## 4. Reproduce Process (random seed 고정해야하겠네) : 코드 집중
To enhance Trainhandler에 대한 개괄적인 설명을 하고 주석으로 설명되어 있음 디테일한 내용은. We divide our reproduce process into three steps:
1. Training the main network (~`Section 2.2`)
    * `python main.py Config/main.json`
2. Pruning the main network (~`Section 2.3`)
3. Training the early exiting module with the main model frozen (~`Section 2.4`)

## Appendix. More details
* zero validation set: valid를 위하여 투자를 하지 않고 train으로 모두 이용하는 것이 최종 train accuracy에 도움이 되었다.

(+) 작은 model에서 다양한 compression 방법들의 실패기록